In [1]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('hi').getOrCreate()

In [5]:
data=[('eli',30),('el',53)]
df=spark.createDataFrame(data,['name','age'])
df.show()

+----+---+
|name|age|
+----+---+
| eli| 30|
|  el| 53|
+----+---+



In [7]:
df.describe()

DataFrame[summary: string, name: string, age: string]

In [6]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [10]:
df.withColumn('new',df['age']*2).show()

+----+---+---+
|name|age|new|
+----+---+---+
| eli| 30| 60|
|  el| 53|106|
+----+---+---+



In [11]:
df.withColumnRenamed('new','two tiems of age').show(1)

+----+---+
|name|age|
+----+---+
| eli| 30|
+----+---+
only showing top 1 row



In [12]:
df.select('name').show()

+----+
|name|
+----+
| eli|
|  el|
+----+



In [13]:
df.filter(df['age']<60).show()

+----+---+
|name|age|
+----+---+
| eli| 30|
|  el| 53|
+----+---+



In [15]:
df.groupBy('name').count().show()

+----+-----+
|name|count|
+----+-----+
| eli|    1|
|  el|    1|
+----+-----+



In [19]:
df.selectExpr("name as word").show()

+----+
|word|
+----+
| eli|
|  el|
+----+



In [31]:
df.selectExpr("split(name,'(?<=.)') as dou").show()

+-----------+
|        dou|
+-----------+
|[e, l, i, ]|
|   [e, l, ]|
+-----------+



In [29]:
df.selectExpr("explode(split(name,'(?<=.)')) as new").show()

+---+
|new|
+---+
|  e|
|  l|
|  i|
|   |
|  e|
|  l|
|   |
+---+



In [33]:


result_df = df.selectExpr("explode(split(name, '(?<=.)')) as word") \
    .groupBy("word").count()

result_df.show()

+----+-----+
|word|count|
+----+-----+
|   l|    2|
|   e|    2|
|   i|    1|
|    |    2|
+----+-----+



In [47]:
df.selectExpr("explode(split(name,'(?<=.)')) as hg").groupBy('hg').count().orderBy("count").show()

+---+-----+
| hg|count|
+---+-----+
|  i|    1|
|  l|    2|
|  e|    2|
|   |    2|
+---+-----+



In [49]:
df.createOrReplaceTempView("my_table")

In [50]:
spark.sql("select * from my_table").show()

+----+---+
|name|age|
+----+---+
| eli| 30|
|  el| 53|
+----+---+



In [52]:
spark.catalog.dropTempView("my_table")

True

In [54]:
spark.catalog.tableExists("my_table")

False

In [72]:
spark2 = SparkSession.builder.appName("WindowFunctions").getOrCreate()

data = [
    (1, "Alice", "HR", 3000),
    (2, "Bob", "HR", 4000),
    (3, "Charlie", "IT", 5000),
    (4, "David", "IT", 6000),
    (5, "Eve", "IT", 7000),
    (6, "Frank", "Finance", 4500),
    (7, "Grace", "Finance", 5500),
]

df2 = spark2.createDataFrame(data, ["id", "name", "department", "salary"])
df2.show()

+---+-------+----------+------+
| id|   name|department|salary|
+---+-------+----------+------+
|  1|  Alice|        HR|  3000|
|  2|    Bob|        HR|  4000|
|  3|Charlie|        IT|  5000|
|  4|  David|        IT|  6000|
|  5|    Eve|        IT|  7000|
|  6|  Frank|   Finance|  4500|
|  7|  Grace|   Finance|  5500|
+---+-------+----------+------+



In [73]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import row_number, rank, dense_rank, sum, avg, count, lag, lead
wind=Window.partitionBy('department').orderBy('salary')
df2.withColumn('rank',F.rank().over(wind)).show()

+---+-------+----------+------+----+
| id|   name|department|salary|rank|
+---+-------+----------+------+----+
|  6|  Frank|   Finance|  4500|   1|
|  7|  Grace|   Finance|  5500|   2|
|  1|  Alice|        HR|  3000|   1|
|  2|    Bob|        HR|  4000|   2|
|  3|Charlie|        IT|  5000|   1|
|  4|  David|        IT|  6000|   2|
|  5|    Eve|        IT|  7000|   3|
+---+-------+----------+------+----+



In [75]:
#Cumulative salary sum within each department.
df2.withColumn('sum_salary_each_department',sum('salary').over(wind)).show()

+---+-------+----------+------+--------------------------+
| id|   name|department|salary|sum_salary_each_department|
+---+-------+----------+------+--------------------------+
|  6|  Frank|   Finance|  4500|                      4500|
|  7|  Grace|   Finance|  5500|                     10000|
|  1|  Alice|        HR|  3000|                      3000|
|  2|    Bob|        HR|  4000|                      7000|
|  3|Charlie|        IT|  5000|                      5000|
|  4|  David|        IT|  6000|                     11000|
|  5|    Eve|        IT|  7000|                     18000|
+---+-------+----------+------+--------------------------+



In [70]:
spark2.stop()